### xgboost model to predict mean player outcomes
used in conjunction with nn algs to determine mean wrc for player. Three diffrent models based on available data then combinded to create an xgb predection.

In [79]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

os.chdir('C:/Users/dalto/OneDrive/Pictures/Documents/Projects/MLB Season Model/')

In [80]:
# import data
data_02_24 = pd.read_csv('./data/cleaned/data_02_24.csv')
df = data_02_24.copy() # import data
df = df.drop(columns=['Unnamed: 0'])

#### find the z_score of each stat
weighted avg of year, std of year, 10 away from 100 is one std


In [81]:
def mean_year(group):
        avg = np.average(group, weights=group.index.map(lambda x: df.loc[x, 'PA']))
        return avg

In [82]:
# weighted std to account for low PA outlier preformances
def calculate_weighted_std(group):
    avg = np.average(group, weights=group.index.map(lambda x: df.loc[x, 'PA']))
    variance = np.average((group - avg)**2, weights=group.index.map(lambda x: df.loc[x, 'PA']))
    return np.sqrt(variance)

In [83]:
def z_scores(player_df, mean_df, std_df):
    # z scores for each stat and player, also creating a new df flow, add back names based on MLBAMID later
    z_scores = pd.DataFrame(index=player_df.index, columns=player_df.select_dtypes(include=[np.number]).columns)

    # scaled for each col
    for col in player_df.select_dtypes(include=[np.number]).columns:
        if col not in ['Season', 'MLBAMID']:  # not needed
            for idx in player_df.index:
                season = player_df.loc[idx, 'Season']
                value = player_df.loc[idx, col]
                mean = mean_df.loc[mean_df['Season'] == season, col].iloc[0]
                std = std_df.loc[std_df['Season'] == season, col].iloc[0]
                # scale so that 10 is 1 std away
                z_scores.loc[idx, col] = 100 + ((value - mean) / std * 10)

    z_scores['Season'] = player_df['Season']
    
    # cols to numeric
    for col in z_scores.columns:
        if col not in ['Season', 'MLBAMID']:
            z_scores[col] = pd.to_numeric(z_scores[col])
    
    # reinsert player names
    z_scores.insert(0, 'Name', player_df['Name']) 

    return z_scores

In [84]:
# df to store mean results
mean_values = pd.DataFrame(index=df['Season'].unique())

for col in df.select_dtypes(include=[np.number]).columns:
    if col != 'Season' and col != 'MLBAMID':  # skip meaningless numeric
        yearly_mean = df.groupby('Season')[col].apply(mean_year)
        mean_values[col] = yearly_mean

# reset and rename index
mean_values = mean_values.reset_index()
mean_values = mean_values.rename(columns={'index': 'Season'})

# sort and display
mean_values = mean_values.sort_values(by=['Season'])
mean_values.head()

,Season,PA,Age,BB%,K%,BABIP,wRC+,BsR,Off,Def,WAR,O-Swing%,Z-Swing%,O-Contact%,Z-Contact%,Contact%,SwStr%,CSW%
19,2002,473.684181,29.200233,0.088931,0.162881,0.294877,99.856949,0.221509,3.922156,-0.767819,1.888873,0.181982,0.703447,0.507012,0.860107,0.797336,0.094299,0.256508
22,2003,475.461939,29.191111,0.086575,0.159114,0.296402,99.927082,0.141683,4.016513,-0.608826,1.909976,0.221738,0.691527,0.527634,0.882055,0.800120,0.093138,0.260791
20,2004,477.651063,29.335336,0.087845,0.163399,0.299236,99.906874,0.255112,4.303784,-0.517574,1.945867,0.204546,0.693372,0.623672,0.855852,0.806547,0.088749,0.255794
14,2005,475.451649,29.259353,0.083354,0.159429,0.297611,100.116588,0.277356,3.656253,-0.893002,1.858494,0.214757,0.681891,0.550610,0.886206,0.813783,0.086017,0.255201
15,2006,483.988528,29.191618,0.085995,0.162801,0.303676,100.090682,0.168365,3.574442,-0.760455,1.862631,0.232215,0.671328,0.584224,0.889358,0.816041,0.084597,0.251676


In [85]:
# Create empty DataFrame to store results
std_values = pd.DataFrame(index=df['Season'].unique())

# Loop through numeric columns
for col in df.select_dtypes(include=[np.number]).columns:
    if col != 'Season' and col != 'MLBAMID':  # skip meaningless numeric
        yearly_std = df.groupby('Season')[col].apply(calculate_weighted_std)
        std_values[col] = yearly_std

std_values = std_values.reset_index()
std_values = std_values.rename(columns={'index': 'Season'})

std_values = std_values.sort_values(by=['Season'])
std_values.head()

,Season,PA,Age,BB%,K%,BABIP,wRC+,BsR,Off,Def,WAR,O-Swing%,Z-Swing%,O-Contact%,Z-Contact%,Contact%,SwStr%,CSW%
19,2002,184.243302,4.009692,0.037986,0.055977,0.035674,31.645660,3.121745,19.286650,9.129972,2.195603,0.047095,0.056035,0.108927,0.051809,0.061293,0.032167,0.028209
22,2003,185.106060,4.078477,0.035647,0.053696,0.034696,30.915634,3.282536,17.866265,9.428022,2.134635,0.048461,0.058644,0.107443,0.047224,0.060704,0.031931,0.027495
20,2004,189.820910,4.146485,0.037057,0.057978,0.036772,30.199233,3.575905,17.922925,9.712096,2.177108,0.051101,0.059314,0.106975,0.055553,0.063824,0.033339,0.030215
14,2005,182.884548,4.155987,0.032447,0.054885,0.035186,28.238546,3.328612,16.513000,10.060394,1.953790,0.057376,0.059317,0.117271,0.047543,0.061264,0.032130,0.027910
15,2006,184.138008,4.310582,0.035272,0.055416,0.036208,28.100105,3.600161,16.729148,9.906593,1.979241,0.060376,0.059520,0.112736,0.046223,0.061019,0.031551,0.028022


apply z scores

In [86]:
z_score_df = z_scores(player_df=df, mean_df=mean_values, std_df=std_values)
z_score_df.head()

,Name,Season,PA,Age,BB%,K%,BABIP,wRC+,BsR,Off,Def,WAR,O-Swing%,Z-Swing%,O-Contact%,Z-Contact%,Contact%,SwStr%,CSW%
0,brent clevlen,2007,74.826870,85.958971,74.686219,193.060161,107.344122,43.282571,98.653170,96.363387,100.657718,89.901996,138.703183,88.950287,71.666459,22.211544,37.187201,169.893575,204.596460
1,brett hayes,2009,74.297907,90.367791,72.475999,127.859623,108.219882,117.931506,99.866315,98.235778,100.843512,91.580480,170.668602,80.735141,60.421763,105.381033,65.578050,147.469352,199.428878
2,carlos peguero,2014,75.723755,96.257739,107.348483,147.612705,148.436984,92.738037,100.377615,97.711317,100.971051,91.256604,122.457976,73.452584,41.259125,91.336666,33.462464,156.578380,183.278087
3,seby zavala,2019,75.887131,92.057094,73.040518,181.646207,108.098997,44.146682,99.594613,95.874994,101.729855,90.245625,129.458257,111.272068,55.708080,34.476706,40.158323,179.134016,179.993300
4,franklin barreto,2020,77.834198,88.981943,76.647649,142.109822,78.737137,59.956124,101.610132,90.946521,99.887662,86.747334,120.942395,87.572409,55.233155,84.442687,56.517822,144.969583,165.941780


### xgboost model

In [87]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

make wrc+ value the next year

In [88]:
# remove 2024 to train model bc we do not know the following year
train_z = z_score_df[z_score_df['Season'] < 2024].copy()
train_z['next_wrc+'] = np.nan # empty col


for idx in train_z.index:
    # retrive name, season
    name = train_z.loc[idx, 'Name']
    season = train_z.loc[idx, 'Season']
    # full df
    next_year_noscale = df[(df['Name'] == name) & (df['Season'] == season + 1)]
    next_year_scaled = z_score_df[(z_score_df['Name'] == name) & (z_score_df['Season'] == season + 1)]

    # add to df
    if not next_year_noscale.empty:
        train_z.loc[idx, 'next_wrc+'] = next_year_noscale['wRC+'].iloc[0]
        train_z.loc[idx, 'next_pa'] = next_year_scaled['PA'].iloc[0] # need for later anaylsis

# remove rows of nas (player retired or smth)
train_z = train_z.dropna(subset=['next_wrc+'])

# display
train_z.head()

,Name,Season,PA,Age,BB%,K%,BABIP,wRC+,BsR,Off,...,WAR,O-Swing%,Z-Swing%,O-Contact%,Z-Contact%,Contact%,SwStr%,CSW%,next_wrc+,next_pa
0,brent clevlen,2007,74.826870,85.958971,74.686219,193.060161,107.344122,43.282571,98.653170,96.363387,...,89.901996,138.703183,88.950287,71.666459,22.211544,37.187201,169.893575,204.596460,40.620897,76.461476
1,brett hayes,2009,74.297907,90.367791,72.475999,127.859623,108.219882,117.931506,99.866315,98.235778,...,91.580480,170.668602,80.735141,60.421763,105.381033,65.578050,147.469352,199.428878,68.663763,79.520422
2,carlos peguero,2014,75.723755,96.257739,107.348483,147.612705,148.436984,92.738037,100.377615,97.711317,...,91.256604,122.457976,73.452584,41.259125,91.336666,33.462464,156.578380,183.278087,92.949027,79.756145
5,domingo santana,2014,76.161502,80.937328,92.701724,191.474452,27.049843,35.300547,100.388715,95.262033,...,88.346507,104.533169,117.006084,37.346862,46.681858,34.973860,163.064364,174.135337,111.757590,85.107766
8,eliezer alfonzo,2008,75.550163,100.406271,73.751790,132.428726,56.320994,41.119256,98.386518,95.994778,...,89.755634,150.740042,83.057646,74.748966,56.559446,48.895200,160.671646,180.916565,19.347656,80.120523


In [89]:
train_z.corr(numeric_only=True)

,Season,PA,Age,BB%,K%,BABIP,wRC+,BsR,Off,Def,WAR,O-Swing%,Z-Swing%,O-Contact%,Z-Contact%,Contact%,SwStr%,CSW%,next_wrc+,next_pa
Season,1.000000,0.014546,-0.019569,0.029076,-0.008916,0.022997,0.018870,0.021108,0.002037,0.008562,0.007766,-0.047145,-0.026187,-0.006109,0.003512,0.004835,-0.018717,0.004084,0.000939,0.015591
PA,0.014546,1.000000,0.191096,0.199412,-0.343358,0.193752,0.479191,0.069551,0.407104,-0.064043,0.670455,-0.123830,0.005106,0.224908,0.211138,0.242266,-0.250521,-0.316426,0.361969,0.654934
Age,-0.019569,0.191096,1.000000,0.134701,-0.181553,-0.057687,0.090597,-0.173281,0.015743,-0.099561,0.044732,-0.135402,-0.039985,0.101000,0.118745,0.144529,-0.161465,-0.152785,-0.017348,-0.030097
BB%,0.029076,0.199412,0.134701,1.000000,0.022289,0.040331,0.403509,-0.035738,0.386002,-0.152251,0.299057,-0.629797,-0.256730,-0.095223,-0.093689,-0.039231,-0.178478,-0.048652,0.219980,0.166223
K%,-0.008916,-0.343358,-0.181553,0.022289,1.000000,-0.001656,-0.260139,0.004901,-0.098615,-0.062773,-0.237804,0.113298,0.110484,-0.715728,-0.752830,-0.840315,0.763811,0.707644,-0.107718,-0.275708
BABIP,0.022997,0.193752,-0.057687,0.040331,-0.001656,1.000000,0.643368,0.091450,0.343506,-0.033747,0.316834,-0.011984,0.028817,-0.014869,-0.015361,-0.021388,0.018797,-0.007627,0.109904,0.174280
wRC+,0.018870,0.479191,0.090597,0.403509,-0.260139,0.643368,1.000000,-0.002380,0.673244,-0.148332,0.625416,-0.226157,0.008656,0.086486,0.077121,0.105670,-0.164731,-0.245755,0.345374,0.427962
BsR,0.021108,0.069551,-0.173281,-0.035738,0.004901,0.091450,-0.002380,1.000000,0.182185,0.171864,0.238929,-0.052483,-0.109870,0.032315,0.037251,0.046845,-0.054323,0.060881,-0.002778,0.078367
Off,0.002037,0.407104,0.015743,0.386002,-0.098615,0.343506,0.673244,0.182185,1.000000,-0.196865,0.819123,-0.182267,0.016900,0.004441,-0.010294,0.008665,-0.063911,-0.147038,0.382569,0.407933
Def,0.008562,-0.064043,-0.099561,-0.152251,-0.062773,-0.033747,-0.148332,0.171864,-0.196865,1.000000,0.271187,0.041378,-0.054239,0.092516,0.086778,0.103715,-0.073243,-0.003418,-0.105743,-0.023805


In [90]:
X = train_z.drop(columns=['Name', 'Season', 'BsR', 'Def', 'next_wrc+', 'O-Contact%','Z-Swing%', 'O-Swing%'])
Y = train_z['next_wrc+']
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.25, random_state=26)

In [91]:
model = xgb.XGBRegressor(n_jobs=-1, n_estimators=50000, device = "cuda", eval_metric = ['mae', 'rmse']) 

##### hyper para tuning

random search to narrow space into grid search

In [92]:
from sklearn.model_selection import RandomizedSearchCV

In [93]:
rnd_search_params = {
    'learning_rate': np.linspace(0.05, 0.2, 4),          
    'max_leaves': np.linspace(1, 20, 10, dtype=int),     
    'min_child_weight': np.linspace(1, 10, 10),         
    'subsample': np.linspace(0.3, 0.9, 7),               
    'colsample_bytree': np.linspace(0.4, 1, 7),          
    'early_stopping_rounds': np.linspace(10, 100, 10, dtype=int),  
    'max_depth': np.linspace(0, 20, 11, dtype=int)       
}

rnd_searcher = RandomizedSearchCV(model, rnd_search_params, random_state=26, n_iter=100, cv=10, n_jobs=-1, verbose=4)
rnd_searcher.fit(xtrain, ytrain, eval_set=[(xtest, ytest)])
print(rnd_searcher.best_params_)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
[0]	validation_0-mae:28.09850	validation_0-rmse:38.01264
[1]	validation_0-mae:27.61500	validation_0-rmse:37.45615
[2]	validation_0-mae:27.18426	validation_0-rmse:36.94628
[3]	validation_0-mae:26.80130	validation_0-rmse:36.47984
[4]	validation_0-mae:26.44859	validation_0-rmse:36.05297
[5]	validation_0-mae:26.21985	validation_0-rmse:35.80519
[6]	validation_0-mae:26.02980	validation_0-rmse:35.59937
[7]	validation_0-mae:25.71826	validation_0-rmse:35.23244
[8]	validation_0-mae:25.53517	validation_0-rmse:35.03823
[9]	validation_0-mae:25.26023	validation_0-rmse:34.71952
[10]	validation_0-mae:25.12007	validation_0-rmse:34.57261
[11]	validation_0-mae:24.87972	validation_0-rmse:34.28413
[12]	validation_0-mae:24.65994	validation_0-rmse:34.02284
[13]	validation_0-mae:24.52284	validation_0-rmse:33.88419
[14]	validation_0-mae:24.33403	validation_0-rmse:33.65768
[15]	validation_0-mae:24.15455	validation_0-rmse:33.44990
[16]	validation_0

c:\Users\dalto\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [14:54:35] WARNING: C:\b\abs_90_bwj_86a\croot\xgboost-split_1724073762025\work\src\context.cc:196: XGBoost is not compiled with CUDA support.
  warnings.warn(smsg, UserWarning)


[20]	validation_0-mae:23.44496	validation_0-rmse:32.64899
[21]	validation_0-mae:23.32256	validation_0-rmse:32.51774
[22]	validation_0-mae:23.21433	validation_0-rmse:32.40608
[23]	validation_0-mae:23.10422	validation_0-rmse:32.29410
[24]	validation_0-mae:23.01762	validation_0-rmse:32.20064
[25]	validation_0-mae:22.94935	validation_0-rmse:32.13897
[26]	validation_0-mae:22.87112	validation_0-rmse:32.05915
[27]	validation_0-mae:22.79311	validation_0-rmse:31.97945
[28]	validation_0-mae:22.71815	validation_0-rmse:31.90544
[29]	validation_0-mae:22.66648	validation_0-rmse:31.84381
[30]	validation_0-mae:22.61370	validation_0-rmse:31.78599
[31]	validation_0-mae:22.56844	validation_0-rmse:31.73059
[32]	validation_0-mae:22.51809	validation_0-rmse:31.67857
[33]	validation_0-mae:22.47243	validation_0-rmse:31.63309
[34]	validation_0-mae:22.43421	validation_0-rmse:31.59849
[35]	validation_0-mae:22.39732	validation_0-rmse:31.56232
[36]	validation_0-mae:22.35950	validation_0-rmse:31.52338
[37]	validatio

grid search

In [94]:
from sklearn.model_selection import GridSearchCV
grid_search_params = {
    'learning_rate': [0.05, 1],   
    'max_leaves': [9, 8, 10],
    'min_child_weight': [1, 2, 3],      
    'subsample': [0.9, 0.95, 1],            
    'colsample_bytree':[0.8, 0.9],     
    'early_stopping_rounds':[70, 80],
    'max_depth': [1, 2, 3]
}

# take subset for faster convergence
xtrain_sub = xtrain.sample(4000)
ytrain_sub = []
for idx in xtrain_sub.index:
    ytrain_sub.append(ytrain[idx])

In [95]:
grid_searcher = GridSearchCV(model, grid_search_params, cv=5, n_jobs=-1, verbose=4)
grid_searcher.fit(xtrain_sub, ytrain_sub, eval_set=[(xtest, ytest)])
print(grid_searcher.best_params_)

Fitting 5 folds for each of 648 candidates, totalling 3240 fits


c:\Users\dalto\anaconda3\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


[0]	validation_0-mae:28.21972	validation_0-rmse:38.16154


c:\Users\dalto\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [14:56:55] WARNING: C:\b\abs_90_bwj_86a\croot\xgboost-split_1724073762025\work\src\context.cc:196: XGBoost is not compiled with CUDA support.
  warnings.warn(smsg, UserWarning)


[1]	validation_0-mae:27.90414	validation_0-rmse:37.75547
[2]	validation_0-mae:27.54629	validation_0-rmse:37.36139
[3]	validation_0-mae:27.28682	validation_0-rmse:37.01641
[4]	validation_0-mae:26.98603	validation_0-rmse:36.68064
[5]	validation_0-mae:26.79418	validation_0-rmse:36.40228
[6]	validation_0-mae:26.61908	validation_0-rmse:36.21260
[7]	validation_0-mae:26.36382	validation_0-rmse:35.92921
[8]	validation_0-mae:26.21380	validation_0-rmse:35.70156
[9]	validation_0-mae:25.97619	validation_0-rmse:35.44868
[10]	validation_0-mae:25.83214	validation_0-rmse:35.29436
[11]	validation_0-mae:25.70624	validation_0-rmse:35.10318
[12]	validation_0-mae:25.48685	validation_0-rmse:34.88323
[13]	validation_0-mae:25.36835	validation_0-rmse:34.75386
[14]	validation_0-mae:25.24813	validation_0-rmse:34.58521
[15]	validation_0-mae:25.10922	validation_0-rmse:34.40585
[16]	validation_0-mae:24.92191	validation_0-rmse:34.22361
[17]	validation_0-mae:24.83405	validation_0-rmse:34.09538
[18]	validation_0-mae:2

In [96]:
opti_para = {'colsample_bytree': 0.9, 'early_stopping_rounds': 70, 'learning_rate': 0.05, 'max_depth': 2, 'max_leaves': 10, 'min_child_weight': 1, 'subsample': 1}
# update model parameters
model.set_params(**opti_para)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9, device='cuda', early_stopping_rounds=70,
             enable_categorical=False, eval_metric=['mae', 'rmse'],
             feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.05, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=2,
             max_leaves=10, min_child_weight=1, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=50000,
             n_jobs=-1, num_parallel_tree=None, random_state=None, ...)

xgb.cv for detailed analysis, optimal boosting rounds

In [97]:
matrix_data = xgb.DMatrix(X, Y)

fmodel = xgb.cv(opti_para, dtrain=matrix_data, num_boost_round=1000, nfold=10, metrics= ['mae', 'rmse'], as_pandas=True, seed=26, early_stopping_rounds=70)
fmodel.head()

c:\Users\dalto\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [14:56:59] WARNING: C:\b\abs_90_bwj_86a\croot\xgboost-split_1724073762025\work\src\learner.cc:740: 
Parameters: { "early_stopping_rounds" } are not used.

  warnings.warn(smsg, UserWarning)


,train-mae-mean,train-mae-std,train-rmse-mean,train-rmse-std,test-mae-mean,test-mae-std,test-rmse-mean,test-rmse-std
0,28.305315,0.108635,38.397238,0.161482,28.313793,0.932080,38.377957,1.499111
1,27.830003,0.110701,37.814051,0.154851,27.846875,0.915714,37.801869,1.477465
2,27.393592,0.105948,37.274397,0.151096,27.415693,0.910748,37.267636,1.456690
3,26.992764,0.103055,36.777027,0.146309,27.025675,0.900086,36.783047,1.433424
4,26.622865,0.100263,36.317284,0.142166,26.658676,0.893656,36.331425,1.412035


#### opti model

In [ ]:
opti_para = {'colsample_bytree': 0.9, 'early_stopping_rounds': 70, 'learning_rate': 0.05, 'max_depth': 2, 'max_leaves': 10, 'min_child_weight': 1, 'subsample': 1}
# update model parameters
model.set_params(**opti_para)

In [98]:
model.set_params(n_estimators=310, early_stopping_rounds=None)
model.fit(xtrain, ytrain)
ypreds = model.predict(xtest)

c:\Users\dalto\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [14:57:08] WARNING: C:\b\abs_90_bwj_86a\croot\xgboost-split_1724073762025\work\src\context.cc:196: XGBoost is not compiled with CUDA support.
  warnings.warn(smsg, UserWarning)


##### results df

In [99]:
results_df = xtest.copy()
results_df['Name'] = train_z.loc[xtest.index, 'Name']
results_df['Season'] = train_z.loc[xtest.index, 'Season']
results_df['next_pa'] = train_z.loc[xtest.index, 'next_pa']
results_df['pred_wrc+'] = ypreds
results_df['season_plus1_wrc+'] = ytest
results_df = results_df[['Name', 'Season', 'pred_wrc+', 'season_plus1_wrc+', 'next_pa']]
results_df['miss'] = (results_df['season_plus1_wrc+'] - results_df['pred_wrc+']).abs()
results_df.head()

,Name,Season,pred_wrc+,season_plus1_wrc+,next_pa,miss
12473,scooter gennett,2015,94.241753,92.114351,104.160553,2.127402
9093,manny machado,2023,118.092567,121.960891,110.781131,3.868324
9240,barry larkin,2003,85.688385,99.415008,95.171709,13.726623
11548,edwin encarnacion,2011,113.448204,150.162933,110.102242,36.714728
10022,oswaldo cabrera,2023,80.977310,87.606563,93.253068,6.629253


In [100]:
results_df = results_df.sort_values(by='season_plus1_wrc+')
results_df = results_df.sort_values(by='season_plus1_wrc+')

# Filter out extreme values
results_df = results_df[(results_df['season_plus1_wrc+'] < 150) & (results_df['season_plus1_wrc+'] > 50)]

# Calculate correlation
r2Results = results_df['pred_wrc+'].corr(results_df['season_plus1_wrc+'])
print(f"Removing extreme wrc+ r^2 {r2Results}")

Removing extreme wrc+ r^2 0.48696627649828494


#### Metrics

In [101]:
from sklearn.metrics import r2_score, root_mean_squared_error, mean_absolute_error

r2 = r2_score(y_true=ytest, y_pred=ypreds)
mse = mean_absolute_error(y_true=ytest, y_pred=ypreds)
rmse = root_mean_squared_error(y_true=ytest, y_pred=ypreds)


print(f"R^2: {r2} \n mse: {mse} \n rmse: {rmse}")

R^2: 0.36273882383921274 
 mse: 21.413456365144032 
 rmse: 30.79523135166027
